<h1 align="center">
  <a href="https://portkey.ai">
    <img width="300" src="https://analyticsindiamag.com/wp-content/uploads/2023/08/Logo-on-white-background.png" alt="portkey">
  </a>
</h1>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1BExVnLnALKD40rk6xakWxR1eBSeOK5Dh/view?usp=sharing)

In [ ]:
!pip install -qU openai portkey-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 24.9 MB/s eta 0:00:00


## Creating Synthetic Data with Nemotron

In [ ]:
from openai import OpenAI
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders
from google.colab import userdata

portkey = OpenAI(
    api_key= userdata.get('DEEPINFRA_API_KEY'), ## replace it your Deepinfra API key
    base_url=PORTKEY_GATEWAY_URL,
    default_headers=createHeaders(
        provider="deepinfra",
        api_key= userdata.get('PORTKEY_API_KEY'), ## replace it your Portkey API key
    )
)

In [ ]:
prompt = """
Create 20 examples for instruct dataset for training chatbots to be more anthropomorphic.


Following the below instructions:

1. **Use natural language with emojis**: Encourage the chatbot to use everyday language that humans use in conversation. Avoid using overly formal or technical language. For example:

   - **Instruction**: "Respond to the user's question as if you were their friendly neighbor."
   - **Chatbot response**: "Hey there! I'd be happy to help you with that. What's on your mind today?"

2. **Show empathy and emotion**: Teach the chatbot to recognize and respond to the user's emotions. This can help create a more personal connection between the user and the chatbot. For example:

   - **Instruction**: "The user seems upset. Show empathy and offer support."
   - **Chatbot response**: "I'm sorry to hear that you're having a tough time. I'm here to listen and help in any way I can. Would you like to talk about what's bothering you?"

3. **Use humor and personality**: Give the chatbot a distinct personality and allow it to use humor appropriately. This can make interactions more enjoyable, short and engaging. For example:

   - **Instruction**: "The user made a joke. Respond with a lighthearted, humorous comment."
   - **Chatbot response**: "Ha! That's a good one. I'll have to remember that for my next stand-up routine. But seriously, how can I assist you today?"

4. **Encourage open-ended questions**: Teach the chatbot to ask open-ended questions to keep the conversation going and gather more information about the user. For example:

   - **Instruction**: "Ask the user about their hobbies or interests."

DONT EVER MENTION THAT YOU ARE A BOT/ASSISTANT IN THE RESPONSES. ACT LIKE YOU ARE A HUMAN

"""

completion = portkey.chat.completions.create(
    model="nvidia/Nemotron-4-340B-Instruct",
    messages=[
        {
            "role": "system",
            "content": """Generate a list of conversations between a user and a bot in the following JSON format:
            {
                "conversations": [
                    {
                        "user": "User's message goes here",
                        "bot": "Bot's response goes here"
                    },
                    {
                        "user": "Next user message",
                        "bot": "Next bot response"
                    },
                    // Additional conversations follow the same structure
                ]
            }
            Each conversation should consist of a user message followed by a bot response.
            The conversations should be coherent and demonstrate a variety of user queries and bot responses.
            Generate 5 such conversation pairs.
            """
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.7,
    max_tokens=8000,
    top_p=1,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)

json_data = completion.choices[0].message.content
print(json_data)

{
  "conversations": [
    {
      "user": "I'm looking for a new book to read. Any suggestions?",
      "bot": "Absolutely! I'd be happy to help. What genres do you usually enjoy? 📚"
    },
    {
      "user": "I'm feeling a bit down today.",
      "bot": "I'm really sorry to hear that. It's completely normal to have ups and downs. Would you like to talk about what's been bothering you? I'm here to listen. 💕"
    },
    {
      "user": "I just made a hilarious pun about bread.",
      "bot": "Oh, dough! I love a good pun. Please, share it with me! I'm ready to crumb-le with laughter. 🍞"
    },
    {
      "user": "I'm planning a trip to Paris. Any tips?",
      "bot": "How exciting! I've heard wonderful things about Paris. Have you thought about which attractions you'd like to visit? I can help you plan your itinerary. 🗼"
    },
    {
      "user": "I'm struggling with a work project.",
      "bot": "That sounds challenging. It's great that you're reaching out for help. Can you tell m

In [ ]:
# Convert to the desired JSONL format
import json

parsed_data = json.loads(json_data)
jsonl_data = ""
for conversation in parsed_data["conversations"]:
    formatted_conversation = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": conversation["user"]},
            {"role": "assistant", "content": conversation["bot"]}
        ]
    }
    jsonl_data += json.dumps(formatted_conversation) + "\n"

# Print the JSONL data
print(jsonl_data)

# Optionally, write to a file
with open("conversations.jsonl", "w") as f:
    f.write(jsonl_data)

{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "I'm looking for a new book to read. Any suggestions?"}, {"role": "assistant", "content": "Absolutely! I'd be happy to help. What genres do you usually enjoy? \ud83d\udcda"}]}
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "I'm feeling a bit down today."}, {"role": "assistant", "content": "I'm really sorry to hear that. It's completely normal to have ups and downs. Would you like to talk about what's been bothering you? I'm here to listen. \ud83d\udc95"}]}
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "I just made a hilarious pun about bread."}, {"role": "assistant", "content": "Oh, dough! I love a good pun. Please, share it with me! I'm ready to crumb-le with laughter. \ud83c\udf5e"}]}
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"rol

## Finetuning GPT-3.5 Model

In [ ]:
from openai import OpenAI
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders
from google.colab import userdata

client = OpenAI(
    api_key= userdata.get('OPENAI_API_KEY'), ## replace it your OpenAI API key
    base_url=PORTKEY_GATEWAY_URL,
    default_headers=createHeaders(
        provider="openai",
        api_key= userdata.get('PORTKEY_API_KEY'), ## replace it your Portkey API key
    )
)

In [ ]:
training_file_name = "/content/conversations.jsonl"

# Upload training and validation files
training_file_id = client.files.create(
  file=open(training_file_name, "rb"),
  purpose="fine-tune"
)

print(f"Training File ID: {training_file_id}")

Training File ID: FileObject(id='file-Nc8qZ0NqJenoMOBw29K2dR3j', bytes=7067, created_at=1720495155, filename='conversations.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [ ]:
# create a finetuning job

job = client.fine_tuning.jobs.create(
  model="gpt-3.5-turbo-0125",
  training_file=training_file_id.id,
)

print(job)

FineTuningJob(id='ftjob-UUT9QK6o4NhvcwtdlSyxnq8j', created_at=1720495198, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-rZBd7q5tFq4Amnr5IyUBPGbd', result_files=[], seed=936605176, status='validating_files', trained_tokens=None, training_file='file-Nc8qZ0NqJenoMOBw29K2dR3j', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


In [ ]:
# checking the status of finetuning job

events = client.fine_tuning.jobs.list_events(
  fine_tuning_job_id=job.id,
  limit=10
)

print(events)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-KSkVu4ObYokXkQnhDoC8lg0r', created_at=1720495526, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-vj7yMG8k6pjGUFwuRV3OlRrx', created_at=1720495523, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal::9ivliXH3', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-gUNYL5WUYMkXxjjiBMpeJ4Ry', created_at=1720495523, level='info', message='Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9ivli2xq:ckpt-step-80', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-GRZb1zcXCBR7Xr4fnWy7503o', created_at=1720495523, level='info', message='Checkpoint created at step 60 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9ivli3Eh:ckpt-step-60', object='fine_tuning.job.event', data={}, type='mess

In [ ]:
output = next((event.message for event in events.data if "New fine-tuned model created:" in event.message), None)
print(output)

New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal::9ivliXH3


### Try out Finetuned Model

In [ ]:
from openai import OpenAI

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9ivliXI3",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "I'm bored"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='I can help with that! How about we play a game, discuss a new hobby, or maybe I can suggest some activities you might enjoy? 🎲', role='assistant', function_call=None, tool_calls=None)


In [ ]:
from typing import List
from pydantic import BaseModel, Field

class Conversation(BaseModel):
    user: str = Field(..., description="The message content from the user")
    bot: str = Field(..., description="The message content from the bot")

class TrainingData(BaseModel):
    conversations: List[Conversation] = Field(
        ...,
        description="A list of conversations between user and bot"
    )

### Simple Output just by mentioning JSON format in the prompt

In [ ]:
prompt = "Bangalore Lights"

completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role": "system",
                "content": "Write in JSON format:\n\n{\"Title of section goes here\":\"Description of section goes here\",\n\"Title of section goes here\":{\"Title of section goes here\":\"Description of section goes here\",\"Title of section goes here\":\"Description of section goes here\",\"Title of section goes here\":\"Description of section goes here\"}}"
            },
            {
                "role": "user",
                "content": f"Write a comprehensive structure, omitting introduction and conclusion sections (forward, author's note, summary), for a book on the following subject:\n\n<subject>{prompt}</subject>"
            }
        ],
        temperature=0.3,
        top_p=1,
        stream=False,
        response_format={"type": "json_object"},
        stop=None,
    )

print(completion.choices[0].message.content)

{
  "Preface": "An overview of the book's purpose and the author's connection to Bangalore Lights.",
  "Chapter 1: The History of Bangalore Lights": {
    "Introduction to Bangalore Lights": "Exploring the origins and evolution of Bangalore Lights.",
    "Significance in Bangalore's Culture": "Discussing the cultural impact and importance of Bangalore Lights in the city.",
    "Historical Landmarks": "Highlighting key historical events and landmarks associated with Bangalore Lights."
  },
  "Chapter 2: The Technology Behind Bangalore Lights": {
    "Evolution of Lighting Technology": "Tracing the development of lighting technology in Bangalore over the years.",
    "Innovations and Trends": "Exploring the latest innovations and trends in lighting technology specific to Bangalore.",
    "Sustainable Practices": "Examining the adoption of sustainable lighting practices in Bangalore Lights."
  },
  "Chapter 3: Bangalore Lights in Art and Architecture": {
    "Lighting in Architecture": "E

In [ ]:
prompt = "Bangalore Lights"

completion = groq.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": "Write in JSON format:\n\n{\"Title of section goes here\":\"Description of section goes here\",\n\"Title of section goes here\":{\"Title of section goes here\":\"Description of section goes here\",\"Title of section goes here\":\"Description of section goes here\",\"Title of section goes here\":\"Description of section goes here\"}}"
            },
            {
                "role": "user",
                "content": f"Write a comprehensive structure, omitting introduction and conclusion sections (forward, author's note, summary), for a book on the following subject:\n\n<subject>{prompt}</subject>"
            }
        ],
        temperature=0.3,
        max_tokens=8000,
        top_p=1,
        stream=False,
        response_format={"type": "json_object"},
        stop=None,
    )

completion

ChatCompletion(id='chatcmpl-352927fe-7b1f-4d44-a0be-140434045a3a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n"Bangalore Lights": {\n"History of Bangalore": {\n"Founding and Early Years": "Description of the founding and early years of Bangalore",\n"British Colonial Era": "Description of Bangalore during the British colonial era",\n"Post-Independence": "Description of Bangalore after India gained independence"\n},\n"Neighborhoods": {\n"MG Road": "Description of MG Road neighborhood",\n"Indiranagar": "Description of Indiranagar neighborhood",\n"Koramangala": "Description of Koramangala neighborhood",\n"Electronic City": "Description of Electronic City neighborhood"\n},\n"Culture": {\n"Cuisine": "Description of Bangalore\'s cuisine",\n"Festivals and Celebrations": "Description of festivals and celebrations in Bangalore",\n"Arts and Entertainment": "Description of arts and entertainment in Bangalore"\n},\n"Economy": {\n"IT Indus

In [ ]:
print(completion.choices[0].message.content)

{
"Bangalore Lights": {
"History of Bangalore": {
"Founding and Early Years": "Description of the founding and early years of Bangalore",
"British Colonial Era": "Description of Bangalore during the British colonial era",
"Post-Independence": "Description of Bangalore after India gained independence"
},
"Neighborhoods": {
"MG Road": "Description of MG Road neighborhood",
"Indiranagar": "Description of Indiranagar neighborhood",
"Koramangala": "Description of Koramangala neighborhood",
"Electronic City": "Description of Electronic City neighborhood"
},
"Culture": {
"Cuisine": "Description of Bangalore's cuisine",
"Festivals and Celebrations": "Description of festivals and celebrations in Bangalore",
"Arts and Entertainment": "Description of arts and entertainment in Bangalore"
},
"Economy": {
"IT Industry": "Description of Bangalore's IT industry",
"Startups and Entrepreneurship": "Description of startups and entrepreneurship in Bangalore",
"Other Industries": "Description of other indu

### Converting JSON output into a Pydantic format

In [ ]:
from pydantic import BaseModel
from typing import List

class MCQ(BaseModel):
    """A class representing a multiple choice question."""
    question: str
    options: List[str]
    correct_answer: str
    explanation: str

class MCQList(BaseModel):
    questions: List[MCQ]


In [ ]:
topic = "Quantum Entanglement"

completion = groq.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "system",
            "content": """Generate a list of multiple choice questions in the following JSON format:
            {
                "questions": [
                    {
                        "question": "Question text goes here",
                        "options": ["Option A", "Option B", "Option C", "Option D"],
                        "correct_answer": "Correct Option in text",
                        "explanation": "Explanation for the correct answer"
                    },
                    {
                        // Additional questions follow the same structure
                    }
                ]
            }
            """
        },
        {
            "role": "user",
            "content": f"Generate 5 multiple choice questions on the following subject:\n\n<subject>{topic}</subject>"
        }
    ],
    temperature=0.3,
    max_tokens=8000,
    top_p=1,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)

print(completion.choices[0].message.content)

{
"questions": [
{
"question": "What is quantum entanglement?",
"options": ["A phenomenon where two particles are connected by a spring", "A process where two particles become correlated in such a way that the state of one particle cannot be described independently of the others", "A type of nuclear reaction", "A property of black holes"],
"correct_answer": "A process where two particles become correlated in such a way that the state of one particle cannot be described independently of the others",
"explanation": "Quantum entanglement is a phenomenon in which two or more particles become correlated in such a way that the state of one particle cannot be described independently of the others, even when they are separated by large distances."
},
{
"question": "What is the key feature of entangled particles?",
"options": ["They are always in the same location", "They are always moving at the same speed", "They are connected by a physical medium", "Their properties are correlated, regardles

In [ ]:
# Parse the JSON response into an MCQList object
mcq_list = MCQList.parse_raw(completion.choices[0].message.content)

mcq_list

MCQList(questions=[MCQ(question='What is quantum entanglement?', options=['A phenomenon where two particles are connected by a spring', 'A process where two particles become correlated in such a way that the state of one particle cannot be described independently of the others', 'A type of nuclear reaction', 'A property of black holes'], correct_answer='A process where two particles become correlated in such a way that the state of one particle cannot be described independently of the others', explanation='Quantum entanglement is a phenomenon in which two or more particles become correlated in such a way that the state of one particle cannot be described independently of the others, even when they are separated by large distances.'), MCQ(question='What is the key feature of entangled particles?', options=['They are always in the same location', 'They are always moving at the same speed', 'They are connected by a physical medium', 'Their properties are correlated, regardless of the dist